## Apollon stem analysis
In this notebook analyse the putative stems of Apollon structure. For each stem, I plot the readnumber of sequences with various number of basepairs and I calculate the fold over expected % of sequences with full basepairing.

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import random
from ivstools import constants

In [2]:
yellow = '#f9bc0cff'
blue = '#293250'
plt.rcParams['font.size'] = 12
plt.rcParams['font.family'] = 'Arial'

In [3]:
def simMutagenesis(s, rate=0.21):
    
    bases = ['A', 'T', 'G', 'C']
    
    # Define probabilities of parent base and the mut bases
    p_parent = 1 - rate
    p_mut = round((rate / 3), 2)
    
    # Resulting seq
    seq = ''
    
    for b in s:
        weights = [p_parent if b == bases[i] else p_mut for i in range(len(bases))]
        seq += random.choices(bases, weights=weights, k=1)[0]
    return seq

In [4]:
def number_of_basepairs(sq1, sq2, orientation='++', basepairs=['AT', 'TA', 'GC', 'CG', 'TG', 'GT']):
    if orientation == '++':
        pass
    elif orientation == '-+':
        sq1 = sq1[::-1]
    elif orientation == '+-':
        sq2 = sq2[::-1]
    elif orientation == '--':
        sq1 = sq1[::-1]
        sq2 = sq1[::-1]
        
    bps = 0
    for b1, b2 in zip(sq1, sq2):
        if b1 + b2 in basepairs:
            bps += 1
    return bps

---
### Observed dataset

In [5]:
df_obs = pd.read_csv('/home/kurfurst/Documents/datasets/MV_apollon_reselection.tsv', sep='\t', names=['count', 'seq'])

In [6]:
# Slicing out stem regions from the observed data
df_obs['stem1_p1'] = [s[0:4] for s in df_obs['seq']]
df_obs['stem1_p2'] = [s[37:41] for s in df_obs['seq']]
df_obs['stem2_p1'] = [s[8:12] for s in df_obs['seq']]
df_obs['stem2_p2'] = [s[29:33] for s in df_obs['seq']]
df_obs['stem3_p1'] = [s[43:51] for s in df_obs['seq']]
df_obs['stem3_p2'] = [s[56:64] for s in df_obs['seq']]
df_obs['bulge_p1'] = [s[4:8] for s in df_obs['seq']]
df_obs['bulge_p2'] = [s[33:37] for s in df_obs['seq']]

In [7]:
# Computing number of basepairs formed in each stem in each sequence
df_obs['stem1_bps'] = [number_of_basepairs(p1, p2, orientation='+-') for p1, p2 in zip(df_obs['stem1_p1'], df_obs['stem1_p2'])]
df_obs['stem2_bps'] = [number_of_basepairs(p1, p2, orientation='+-') for p1, p2 in zip(df_obs['stem2_p1'], df_obs['stem2_p2'])]
df_obs['stem3_bps'] = [number_of_basepairs(p1, p2, orientation='+-') for p1, p2 in zip(df_obs['stem3_p1'], df_obs['stem3_p2'])]
df_obs['bulge_bps'] = [number_of_basepairs(p1, p2, orientation='+-') for p1, p2 in zip(df_obs['bulge_p1'], df_obs['bulge_p2'])]

---
### Simulating the initial dataset

In [ ]:
# Simulating the initial dataset after 21% mutagenesis of the parent sequence
apollon_parent = df.iloc[0]['seq']
mutagenesis_sim = pd.Series([simMutagenesis(apollon_parent, rate=0.21) for x in range(0, df['count'].sum())]).value_counts().sort_values(ascending=False)
df_sim = pd.DataFrame(mutagenesis_sim).reset_index()
df_sim.columns = ['seq', 'count']
df_sim

In [ ]:
# Slicing out stem regions from the simulated data
df_sim['stem1_p1'] = [s[0:4] for s in df_sim['seq']]
df_sim['stem1_p2'] = [s[37:41] for s in df_sim['seq']]
df_sim['stem2_p1'] = [s[8:12] for s in df_sim['seq']]
df_sim['stem2_p2'] = [s[29:33] for s in df_sim['seq']]
df_sim['stem3_p1'] = [s[43:51] for s in df_sim['seq']]
df_sim['stem3_p2'] = [s[56:64] for s in df_sim['seq']]
df_sim['bulge_p1'] = [s[4:8] for s in df_sim['seq']]
df_sim['bulge_p2'] = [s[33:37] for s in df_sim['seq']]

In [ ]:
# Computing number of basepairs formed in each stem in each sequence (in the simulated dataset)
df_sim['stem1_bps'] = [number_of_basepairs(p1, p2, orientation='+-') for p1, p2 in zip(df_sim['stem1_p1'], df_sim['stem1_p2'])]
df_sim['stem2_bps'] = [number_of_basepairs(p1, p2, orientation='+-') for p1, p2 in zip(df_sim['stem2_p1'], df_sim['stem2_p2'])]
df_sim['stem3_bps'] = [number_of_basepairs(p1, p2, orientation='+-') for p1, p2 in zip(df_sim['stem3_p1'], df_sim['stem3_p2'])]
df_sim['bulge_bps'] = [number_of_basepairs(p1, p2, orientation='+-') for p1, p2 in zip(df_sim['bulge_p1'], df_sim['bulge_p2'])]

In [ ]:
df_sim.to_csv('simulated_initial_pool.csv')

#### READ the simulated dataset

In [8]:
df_sim = pd.read_csv('simulated_initial_pool.csv', index_col=0)

---

In [ ]:
stem1_obs = df_obs[df_obs['stem1_bps'] == 4]['count'].sum() / df_obs['count'].sum()
stem1_sim = df_sim[df_sim['stem1_bps'] == 4]['count'].sum() / df_sim['count'].sum()

In [ ]:
stem2_obs = df_obs[df_obs['stem2_bps'] == 4]['count'].sum() / df_obs['count'].sum()
stem2_sim = df_sim[df_sim['stem2_bps'] == 4]['count'].sum() / df_sim['count'].sum()

In [ ]:
stem3_obs = df_obs[df_obs['stem3_bps'] == 8]['count'].sum() / df_obs['count'].sum()
stem3_sim = df_sim[df_sim['stem3_bps'] == 8]['count'].sum() / df_sim['count'].sum()

In [ ]:
pd.DataFrame([
    [stem1_obs, stem1_sim],
    [stem2_obs, stem2_sim],
    [stem3_obs, stem3_sim]
], columns=['obs', 'sim'], index=['stem1', 'stem2', 'stem3']).to_csv('fold_over_expected_values_READS.csv')

In [ ]:
plt.figure(figsize=(3.5, 2.5))
plt.bar(['stem 1', 'stem 2', 'stem 3'], [stem1_obs/stem1_sim, stem2_obs/stem2_sim, stem3_obs/stem3_sim], color=yellow)
plt.ylabel('Fold over expected')
plt.title('Reads')
plt.savefig('fold_over_expected_READS.png', dpi=120)

In [ ]:
stem1_obs = len(df_obs[df_obs['stem1_bps'] == 4]) / len(df_obs)
stem1_sim = len(df_sim[df_sim['stem1_bps'] == 4]) / len(df_sim)

In [ ]:
stem2_obs = len(df_obs[df_obs['stem2_bps'] == 4]) / len(df_obs)
stem2_sim = len(df_sim[df_sim['stem2_bps'] == 4]) / len(df_sim)

In [ ]:
stem3_obs = len(df_obs[df_obs['stem3_bps'] == 8]) / len(df_obs)
stem3_sim = len(df_sim[df_sim['stem3_bps'] == 8]) / len(df_sim)

In [ ]:
pd.DataFrame([
    [stem1_obs, stem1_sim],
    [stem2_obs, stem2_sim],
    [stem3_obs, stem3_sim]
], columns=['obs', 'sim'], index=['stem1', 'stem2', 'stem3']).to_csv('fold_over_expected_values_UNIQUE.csv')

In [ ]:
plt.figure(figsize=(3.5, 2.5))
plt.bar(['stem 1', 'stem 2', 'stem 3'], [stem1_obs/stem1_sim, stem2_obs/stem2_sim, stem3_obs/stem3_sim], color=yellow)
plt.ylabel('Fold over expected')
plt.title('Unique sequences')
plt.savefig('fold_over_expected_UNIQUE.png', dpi=120)

In [ ]:
df_obs['p'] = [s[48] + s[58] for s in df_obs['seq']]

**Reads**

In [ ]:
df_obs[df_obs['p'].isin(['AT', 'TA', 'CG', 'GC', 'TG', 'GT'])]['count'].sum()

In [ ]:
df_obs['count'].sum()

**Unique sequences**

In [ ]:
len((df_obs[df_obs['p'].isin(['AT', 'TA', 'CG', 'GC', 'TG', 'GT'])]))

In [ ]:
len(df_obs)

 ---
### WARNING
**Here I'm appending tree artificial rows just to adjust the scale the following 4 figures. These must not be taken into account for the figures after that!**

In [ ]:
df.columns

In [ ]:
insert_row1 = {'count':0,
          'seq':'ATG',
          'stem1_p1':0,
          'stem1_p2':0,
          'stem2_p1':0,
          'stem2_p2':0,
          'stem3_p1':0,
          'stem3_p2':0,
          'bulge_p1':0,
          'bulge_p2':0,
          'stem1_bps':0,
          'stem2_bps':0,
          'stem3_bps':0,
          'bulge_bps':0
          }

In [ ]:
insert_row2 = {'count':0,
          'seq':'ATG',
          'stem1_p1':0,
          'stem1_p2':0,
          'stem2_p1':0,
          'stem2_p2':0,
          'stem3_p1':0,
          'stem3_p2':0,
          'bulge_p1':0,
          'bulge_p2':0,
          'stem1_bps':0,
          'stem2_bps':0,
          'stem3_bps':1,
          'bulge_bps':0
          }

In [ ]:
insert_row3 = {'count':0,
          'seq':'ATG',
          'stem1_p1':0,
          'stem1_p2':0,
          'stem2_p1':0,
          'stem2_p2':0,
          'stem3_p1':0,
          'stem3_p2':0,
          'bulge_p1':0,
          'bulge_p2':0,
          'stem1_bps':0,
          'stem2_bps':0,
          'stem3_bps':1,
          'bulge_bps':4
          }

In [ ]:
df = pd.concat([df, pd.DataFrame([insert_row1])])
df = pd.concat([df, pd.DataFrame([insert_row2])])
df = pd.concat([df, pd.DataFrame([insert_row3])])

In [ ]:
df.tail()

In [ ]:
sns.stripplot(data=df, x='stem1_bps', y='count', color=yellow, jitter=0.2)
#plt.xticks([0, 1, 2, 3, 4], [0, 1, 2, 3, 4])
plt.yscale('log')
plt.title('Stem 1')
plt.xlabel('Number of basepairs')
plt.ylabel('Read Number')
plt.savefig('stem1_rn.png', dpi=120)

In [ ]:
sns.stripplot(data=df, x='stem2_bps', y='count', color=yellow, jitter=0.2)
plt.yscale('log')
plt.title('Stem 2')
plt.xlabel('Number of basepairs')
plt.ylabel('Read Number')
plt.savefig('stem2_rn.png', dpi=120)

In [ ]:
sns.stripplot(data=df, x='stem3_bps', y='count', color=yellow, jitter=0.2)
plt.yscale('log')
plt.title('Stem 3')
plt.xlabel('Number of basepairs')
plt.ylabel('Read Number')
plt.savefig('stem3_rn.png', dpi=120)

In [ ]:
sns.stripplot(data=df, x='bulge_bps', y='count', color=blue, jitter=0.2)
plt.yscale('log')
plt.title('Bulge')
plt.xlabel('Number of basepairs')
plt.ylabel('Read Number')
plt.savefig('bulge_rn.png', dpi=120)

---
**Stem 1**

In [ ]:
stem1_p = len(df[df['stem1_bps'] == 4])/len(df) * 100
stem1_t = (6/16)**4 * 100

In [ ]:
print('% of sequences with 4/4 basepairs:', round(stem1_p, 2))
print('% theoretical:', round(stem1_t, 2))

---
**Stem 2**

In [ ]:
stem2_p = len(df[df['stem2_bps'] == 4])/len(df) * 100
stem2_t = (6/16)**4 * 100

In [ ]:
print('% of sequences with 4/4 basepairs:', round(stem2_p, 2))
print('% theoretical:', round(stem2_t, 2))

---
**Stem 3**

In [ ]:
stem3_p = len(df[df['stem3_bps'] == 8])/len(df) * 100
stem3_t = (6/16)**8 * 100

In [ ]:
print('% of sequences with 8/8 basepairs:', round(stem3_p, 2))
print('% theoretical:', round(stem3_t, 2))

---
**Bulge**

In [ ]:
bulge_p = len(df[df['bulge_bps'] == 4])/len(df) * 100
bulge_t = (6/16)**4 * 100

In [ ]:
print('% of sequences with 4/4 basepairs:', bulge_p)
print('% theoretical:', round(bulge_t, 2))

---

In [ ]:
plt.bar(['stem 1', 'stem 2', 'stem 3', 'bulge'], [stem1_p/stem1_t, stem2_p/stem2_t, stem3_p/stem3_t, bulge_p/bulge_t], color=yellow)
plt.ylabel('Fold over expected')
plt.savefig('fold_over_expected', dpi=120)